# A script to cumput LLM feedback for prompt enriching task

In [2]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Thesis/MedTransNet

import sys
sys.path.append('/content/drive/MyDrive/Thesis/MedTransNet')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Thesis/MedTransNet
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Thesis/MedTransNet


In [3]:
!pip install protobuf==3.20.0 icetk cpm_kernels torch-geometric datasets
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip -q install bitsandbytes accelerate xformers einops

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 2.1.2 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.1.2 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.1.2 which is incompatible.
torchvision 0.16.0+cu121 requires torch==2.1.0, but you have torch 2.1.2 which is incompatible.


## Setup

In [4]:
import torch
import gc
import pickle
import os

import pandas as pd

from tqdm import tqdm
from datasets import load_dataset

import transformers
from transformers import AutoTokenizer
from transformers import logging as hf_logging

from src.medical_hgt.llm import LLM, LLMFeedback
from config import ROOT_DIR

## LLM

## Inference Loop

In [10]:
def compute_llm_feedback(llm, qa_dataset, nx_graph_data, question_to_subgraphs_mapping):
    correct_answer_map = {0: 'A', 1: 'B', 2: 'C', 3: 'D'}
    llm_feedbacks_dict = {}  # a dict of dicts in the form {node_type_0: {node_index_0: conf_diff_0, node_index_1: conf_diff_1...}, ...}

    for i, (qa_index, subgraph_tuples) in enumerate(tqdm(question_to_subgraphs_mapping.items())):
        qa_row = qa_dataset.iloc[qa_index]
        question_dict = dict(qa_row.drop(['id', 'cop', 'exp']))
        correct_answer = qa_row['cop']
        prompt = """Question: {} A. {} B. {} C. {} D. {}""".format(
            question_dict['question'],
            question_dict['opa'],
            question_dict['opb'],
            question_dict['opc'],
            question_dict['opd']
        )

        # Process question without context
        output_encodings, predictions = llm.inference(prompt)
        llm_vanilla_response_dict = llm.get_confidence(correct_answer_map[correct_answer], output_encodings, predictions)

        if llm_vanilla_response_dict['confidence'] == -1:
            print(f'Wrong response format. Question {i} ignored')
            continue

        # Create LLMFeedback object
        llm_feedback = LLMFeedback(qa_index, llm_vanilla_response_dict['response'], llm_vanilla_response_dict['confidence'], llm_vanilla_response_dict['cop_confidence'], llm_vanilla_response_dict['accuracy'])

        # Batch process contexts

        prompts_with_context = []

        for node_uid, node_type in subgraph_tuples[:20]:
            # Create Context string
            node_name = nx_graph_data.nodes[node_uid]['name']
            context = f'Context: The {node_type} {node_name}. '
            prompt_with_context = context + prompt
            prompts_with_context.append(prompt_with_context)

        # Batch inference
        batch_output_encodings, batch_predictions = llm.inference_batch(prompts_with_context)

        # Postprocess batch model output
        for j, (node_uid, node_type) in enumerate(subgraph_tuples[:20]):
            output_encoding = batch_output_encodings[j]
            prediction = batch_predictions[j]
            llm_context_response_dict = llm.get_confidence(correct_answer_map[correct_answer], output_encoding=output_encoding, predictions=prediction)

            if llm_context_response_dict['confidence'] == -1:
                print(f'Wrong response format. Node {node_uid} ignored')
                continue

            llm_feedback.insert_feedback(node_type, node_uid, llm_context_response_dict['confidence'], llm_context_response_dict['cop_confidence'], llm_context_response_dict['accuracy'])

            llm_feedbacks_dict[qa_index] = llm_feedback

        if i % 100 == 0:
            print(f'Example Feedback:\n')
            llm_feedbacks_dict[qa_index].print_feedback()
            pickle.dump(llm_feedbacks_dict, open(os.path.join(ROOT_DIR, 'datasets', 'train', 'llm_feedback', f'llm_feedbacks_{i}.pickle'), 'wb'))

    return llm_feedbacks_dict

## config and load llm

In [11]:
# 4bit quantization config

bnb_config = transformers.BitsAndBytesConfig(
          load_in_4bit=True,
          bnb_4bit_use_double_quant=True,
          bnb_4bit_quant_type="nf4",
          bnb_4bit_compute_dtype=torch.bfloat16
        )

In [12]:
model_name="mistralai/Mistral-7B-Instruct-v0.1"

In [13]:
model = transformers.AutoModelForCausalLM.from_pretrained( model_name, trust_remote_code=True, quantization_config=bnb_config, device_map='auto')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [15]:
llm = LLM(model, tokenizer)

## Compute feedback

In [16]:
# Load datasets

prime_kg = pickle.load(open(os.path.join(ROOT_DIR, 'datasets/prime_kg_nx_63960.pickle'), 'rb'))

qa_dataset = load_dataset("medmcqa")
qa_dataset = pd.DataFrame(qa_dataset['train'])

subgraphs_dict_path = 'datasets/subgraphs_dict_train.pickle'
question_to_subgraphs_mapping = pickle.load(open(os.path.join(ROOT_DIR, subgraphs_dict_path), 'rb'))

Generating train split:   0%|          | 0/182822 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6150 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4183 [00:00<?, ? examples/s]

In [ ]:
llm_feedbacks_dict = compute_llm_feedback(llm, qa_dataset, prime_kg, question_to_subgraphs_mapping)

pickle.dump(llm_feedbacks_dict, open(os.path.join(ROOT_DIR, 'datasets', f'llm_feedbacks_train_{len(llm_feedbacks_dict)}.pickle'), 'wb'))

 10%|█         | 18427/177004 [00:06<00:56, 2789.04it/s]

Wrong response format. Question 18426 ignored


 11%|█         | 18706/177004 [00:07<01:03, 2495.93it/s]

Wrong response format. Question 18532 ignored


 11%|█         | 18846/177004 [00:07<01:13, 2143.87it/s]

...

 69%|██████▉   | 122200/177004 [23:36:05<48:34:12,  3.19s/it]

Example Feedback:

Response without context: D
Confidence without context: 0.5835819840431213
Cop confidence without context: 0.5835819840431213
LLM is correct without context: True
Confidences with context: {'effect/phenotype': {89276: 0.5136265158653259}, 'gene/protein': {3590: 0.5234652161598206}, 'disease': {33043: 0.5090462565422058}}
Accuracies with context: {'effect/phenotype': {89276: 1}, 'gene/protein': {3590: 1}, 'disease': {33043: 1}}
Cop confidences with context: {'effect/phenotype': {89276: 0.5136265158653259}, 'gene/protein': {3590: 0.5234652161598206}, 'disease': {33043: 0.5090462565422058}}


 69%|██████▉   | 122300/177004 [23:41:57<58:34:28,  3.85s/it]

Example Feedback:

Response without context: B
Confidence without context: 0.2873021960258484
Cop confidence without context: 0.2873021960258484
LLM is correct without context: True
Confidences with context: {'anatomy': {63521: 0.29909247159957886}, 'disease': {36999: 0.3304433822631836, 30152: 0.3106319010257721, 84271: 0.328849732875824, 37594: 0.341179221868515, 37508: 0.3155207335948944}, 'gene/protein': {11011: 0.29304003715515137, 9373: 0.2969866693019867, 10055: 0.29171180725097656, 7563: 0.2920510768890381, 33995: 0.30487263202667236, 4385: 0.2916332185268402, 34001: 0.293792188167572, 6047: 0.28024566173553467}}
Accuracies with context: {'anatomy': {63521: 0}, 'disease': {36999: 0, 30152: 0, 84271: 0, 37594: 0, 37508: 0}, 'gene/protein': {11011: 0, 9373: 0, 10055: 0, 7563: 0, 33995: 0, 4385: 0, 34001: 0, 6047: 0}}
Cop confidences with context: {'anatomy': {63521: 0.24990111589431763}, 'disease': {36999: 0.24365368485450745, 30152: 0.22373977303504944, 84271: 0.2225108742713928

 69%|██████▉   | 122384/177004 [23:46:57<56:38:43,  3.73s/it]